In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
from dateutil.relativedelta import relativedelta
import math
import re
import pyodbc



# Reading Source Data 'ADT_Dummy Data' and 'ADT_ER_IP_04012021to07062021'

server = 'wn000038725' 
database = 'oas_mas_stg' 
#username = 'MS\ssing296' 
#password = 'Rai@garh43' 

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')  
cursor = cnxn.cursor()


#cursor.execute("SELECT TOP(1) * FROM Canopy.ARCH_ADT_ANDOR_ED")

#for row in cursor:  
#    print(row)         

qr1 = "Select TOP(3) * from [Canopy].[ARCH_ADT_ANDOR_ED_input]" 

ADT_data = pd.read_sql(qr1, cnxn)

#print(ADT_data)

df = pd.read_csv(r'\\NAS00782PN\Clients\Payer Presales\Clients\Canopy\Auth and ADT Files from Canopy\CanopyConnect High Risk Codes.csv')

df = df.drop_duplicates(keep='first')

high_cost_codes = pd.read_csv(r'\\NAS00782PN\Clients\Payer Presales\Clients\Canopy\Auth and ADT Files from Canopy\High Cost Chronic Condition Codes.csv')

# filling 'blank_code' in place of null values

ADT_data['Diagnosis'].fillna('blank_code', inplace=True)

# making list of diagnosis, high-risk and high-cost code

diagnosis_lst = ADT_data['Diagnosis'].tolist()

canopy_high_risk_codes = df['code'].tolist()

canopy_high_risk_prim_d = df['Primary Disease'].tolist()

high_cost_chronic_codes = high_cost_codes['code'].tolist()

high_cost_chronic_subcategory = high_cost_codes['Subcategory'].tolist()

text_code = ['abdominal pain','chest pain','back pain','breath','SOB','short of breath','fall','pregnancy','covid']

for i in range(len(canopy_high_risk_codes)):
    if(isinstance(canopy_high_risk_codes[i],int)):
        canopy_high_risk_codes[i]
    else:
        canopy_high_risk_codes[i] = canopy_high_risk_codes[i].lower()

for i in range(len(high_cost_chronic_codes)):
    if(isinstance(high_cost_chronic_codes[i],int)):
        high_cost_chronic_codes[i]
    else:
        high_cost_chronic_codes[i] = high_cost_chronic_codes[i].lower()
        

canopy_high_risk_codes = canopy_high_risk_codes + text_code
high_cost_chronic_codes = high_cost_chronic_codes + text_code

#print(canopy_high_risk_codes)

#cleanedList = [x for x in diagnosis_lst if str(x) != 'nan']

"""

A = ['L sided Dental Pain', 'K0889', 'Dental pain', '30253X1']
B = ['30253G0','30253G1','30253X0','30253X1','30253Y0','30253Y1','30260G0','30260G1']

check1 =  set(A).intersection(B)
print(check1)
if check1:
    print("Chronic Condition")    
else:
    print("Non-Chronic Condition aahe")
"""

def high_risk_cost_flag (diagnosis_lst,canopy_high_risk_codes):
    b = []
    prim_d = []
    codes = []
    for i in range(0,len(diagnosis_lst)):
        a = diagnosis_lst[i]
        split =  a.split(";")
        #print(split)
        clean_lst = []
        
        for i in range(0,len(split)):
            a = split[i].strip()
            a = a.replace(".",'')
            a = a.lower()
            if(re.findall(r"abd*",a)):
                a = "abdominal pain"
            elif(re.findall(r"chest*",a)):
                a = "chest pain"
            elif(re.findall(r"back*",a)):
                a = "back pain"
            elif(re.findall(r"breath*",a) or re.findall(r"sob*",a)):
                a = "short of breath"
            elif(re.findall(r"fall*",a)):
                a = "fall"
            elif(re.findall(r"pregnancy*",a)):
                a = "pregnancy"
            elif(re.findall(r"covid*",a)):
                a = "covid"
            else:
                a
            clean_lst.append(a)

        if set(clean_lst).intersection(canopy_high_risk_codes):
            res = "Y"
            b.append(res)
        else:
            res = "N"
            b.append(res)
        
    #canopy_high_risk_codes['Subcategory'] = canopy_high_risk_codes.code.map(clean_lst)
    #print(b)
    return b

def high_risk_cost_desc (diagnosis_lst,canopy_high_risk_codes,col_name):
    b = []
    prim_d = []
    codes = []
    
    for i in range(0,len(diagnosis_lst)):
        a = diagnosis_lst[i]
        split =  a.split(";")
        #print(split)
        clean_lst = []
        
        for i in range(0,len(split)):
            a = split[i].strip()
            a = a.replace(".",'')
            a = a.lower()
            if(re.findall(r"abd*",a)):
                a = "abdominal pain"
            elif(re.findall(r"chest*",a)):
                a = "chest pain"
            elif(re.findall(r"back*",a)):
                a = "back pain"
            elif(re.findall(r"breath*",a) or re.findall(r"sob*",a)):
                a = "short of breath"
            elif(re.findall(r"fall*",a)):
                a = "fall"
            elif(re.findall(r"pregnancy*",a)):
                a = "pregnancy"
            elif(re.findall(r"covid*",a)):
                a = "covid"
            else:
                a
            clean_lst.append(a)
        
        #for i in (len(diagnosis_lst))
        if set(clean_lst).intersection(canopy_high_risk_codes):
            prim_d = set(clean_lst).intersection(canopy_high_risk_codes)
            #print(set(clean_lst).intersection(canopy_high_risk_codes))
            #prim_d = set(prim_d)
            prim_d = list(prim_d)
            #print(prim_d)
            for i in range(len(prim_d)):
                if(isinstance(prim_d[i],int)):
                    prim_d[i]
                else:
                    prim_d[i] = prim_d[i].upper()
            lsttostr_prim = ','.join([str(elem) for elem in prim_d])
                #print(prim_d)
            
            if(col_name == "Primary Disease"):
                res = df[col_name][df['code'].isin(prim_d)].tolist()
                
            else:
                
                res = high_cost_codes[col_name][high_cost_codes['code'].isin(prim_d)].tolist()
                
            lst_set = set(res)
            #print(res)
            listToStr = ','.join([str(elem) for elem in lst_set])
            #ADT_data["High Risk desc"] = res
            if(len(res) == 0):
                b.append(lsttostr_prim)
            else: 
                b.append(listToStr)
                
        else:
            #print("Non-Chronic Condition")
            res1 = col_name
            res1  =  "Not a High Risk" if(col_name == "Primary Disease") else "Not a High Cost"
            #ADT_data["High Risk desc"] = res
            b.append(res1)
        
    #canopy_high_risk_codes['Subcategory'] = canopy_high_risk_codes.code.map(clean_lst)
    #print(b)
    return b


ADT_data["High_Risk_Flag"] = high_risk_cost_flag(diagnosis_lst,canopy_high_risk_codes)

ADT_data["High_Risk_desc"] = high_risk_cost_desc(diagnosis_lst,canopy_high_risk_codes,"Primary Disease")

ADT_data["High_Cost_Flag"] = high_risk_cost_flag(diagnosis_lst,high_cost_chronic_codes)

ADT_data["High_Cost_desc"] = high_risk_cost_desc(diagnosis_lst,high_cost_chronic_codes,"Subcategory")

#print(ADT_data)

#qry_insert = "Insert into Canopy.ARCH_ADT_ANDOR_ED values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"

qry_insert = "Insert into Canopy.ARCH_ADT_ANDOR_ED (ExternalVisitId,MemberID,DOB,AdmitDate,DischargeDate) values(?,?,?,?,?)"

def check_null(column_value):
    if(pd.isnull(column_value)):
        column_value = None
    else:
        column_value
    return column_value

    

for index,row in ADT_data.iterrows():
    """
    print(row['ExternalVisitId'])
    print(row['MemberID'])
    print(row['DOB'])
    print(row['AdmitDate'])
    print(row['DischargeDate'])
    
    if(pd.isnull(row['DischargeDate'])):
        row['DischargeDate'] = None
    else:
        row['DischargeDate']
    """
    rec = (row['ExternalVisitId'],row['MemberID'],row['DOB'],row['AdmitDate'],check_null(row['DischargeDate']))
    #df=df.append(rec)
    cursor.execute(qry_insert,rec)
    print("Data Load Done")

cnxn.commit()
cursor.close()

Data Load Done
Data Load Done
Data Load Done
